# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data importing
Helper function that will help transforming the string that represents a list in the Excel file, to an actual Python list

In [6]:
def string_to_list(column):
    return [element.strip(" ") for element in column.strip("[]").replace("'", "").replace('"', "").split(", ")]

def single_list_string_to_string(column):
    try:
        return column.replace("'", "").replace('"', "").replace("[", "").replace("]", "")
    except:
        print(column)
        return column

In [7]:
PATH = 'data/'

recensement = pd.read_excel(PATH + '1832_cleaned_data_recensement.xlsx')
recensement["PROPRIÉTAIRES DES MAISONS."] = recensement["PROPRIÉTAIRES DES MAISONS."].apply(string_to_list)
recensement["NOMS DES RUES."] = recensement["NOMS DES RUES."].apply(single_list_string_to_string)
recensement["VOCATION."] = recensement["VOCATION."].apply(single_list_string_to_string)
recensement["PROPRIÉTAIRES DES MAISONS."] = recensement["PROPRIÉTAIRES DES MAISONS."].apply(lambda x : x if len(x[0]) > 0 else None)
recensement["PROPRIÉTAIRES DES MAISONS."] = recensement["PROPRIÉTAIRES DES MAISONS."].fillna(method="ffill")
recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME."] = recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME."].apply(single_list_string_to_string)
recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE."] = recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE."].apply(single_list_string_to_string)

cadastre = pd.read_csv(PATH + 'parcelInfos.csv').query("folio <= 11")
cadastre["nr"] = cadastre["nr"].astype("int32")
cadastre["use"] = cadastre["use"].apply(single_list_string_to_string)

folio_links = pd.read_excel(PATH + "Folio_link.xlsx")

nan
nan
nan


In [ ]:
recensement.head(5)

In [ ]:
cadastre

In [ ]:
folio_links.head(2)

In [8]:
data = []
cadastre["street"] = [[] for _ in range(len(cadastre))]
for i, row in folio_links.iterrows():
    folio = row["folio"]
    for num in range(row["num_start"], row["num_stop"]):
        try:
            for i in range(len(cadastre.loc[(cadastre["nr"] == num) & (cadastre["folio"] == folio), "street"].values)):
                cadastre.loc[(cadastre["nr"] == num) & (cadastre["folio"] == folio), "street"].values[i].append(row["rue"])
        except:
            print(cadastre[(cadastre["nr"] == num) & (cadastre["folio"] == folio)]["street"], folio, num)
cadastre["street"] = cadastre["street"].apply(lambda x : list(set(x)))

In [16]:
cadastre.folio.value_counts()

3     400
8     376
9     289
7     224
6     216
4     215
11    159
2     158
1     149
10    129
5      74
Name: folio, dtype: int64

# Merging the two datasets
Before trying to merge, we must "explode" the recensement dataset such that we get one entry for each owner. Meaning that if an entry has owner [A, B], it should create two entries, the first one belonging to A only and the second one to B only. We then only keep an entry if it has an owner

In [10]:
recensement_exploded = recensement.explode("PROPRIÉTAIRES DES MAISONS.")
recensement_exploded = recensement_exploded[recensement_exploded["PROPRIÉTAIRES DES MAISONS."].str.len() > 0]

Because of this flaw, let's try to split the owner names on blank spaces in the cadastre dataset and explode it as well

In [11]:
cadastre["owner_names"] = cadastre["owner_surname"].apply(lambda name : name.split(" "))
cadastre_exploded = cadastre.explode("owner_names")
cadastre_exploded = cadastre_exploded.explode("street")
cadastre_exploded.loc[30:50]

,Unnamed: 0,Unnamed: 0.1,folio,nr,articles,use,owner_surname,Unnamed: 6,pos,street,owner_names
30,30,30,1,30,NaN,maison,Jaques-Louis Berger,NaN,"[46.51917700375838, 6.631740051334762]",Grand Chêne,Jaques-Louis
30,30,30,1,30,NaN,maison,Jaques-Louis Berger,NaN,"[46.51917700375838, 6.631740051334762]",Petit Chêne,Jaques-Louis
30,30,30,1,30,NaN,maison,Jaques-Louis Berger,NaN,"[46.51917700375838, 6.631740051334762]",Grand Chêne,Berger
30,30,30,1,30,NaN,maison,Jaques-Louis Berger,NaN,"[46.51917700375838, 6.631740051334762]",Petit Chêne,Berger
31,31,31,1,31,NaN,maison,Jaques-Louis Berger,NaN,"[46.519226548862484, 6.631787503526648]",Grand Chêne,Jaques-Louis
...,...,...,...,...,...,...,...,...,...,...,...
49,49,48,1,48,NaN,buanderie,Marie-Antoine De la Forêt,NaN,"[46.51981675515511, 6.63077368419727]",Grand Chêne,Forêt
50,50,49,1,49,NaN,bois,Marie-Antoine De la Forêt,NaN,"[46.52000311197292, 6.63066547811636]",Grand Chêne,Marie-Antoine
50,50,49,1,49,NaN,bois,Marie-Antoine De la Forêt,NaN,"[46.52000311197292, 6.63066547811636]",Grand Chêne,De
50,50,49,1,49,NaN,bois,Marie-Antoine De la Forêt,NaN,"[46.52000311197292, 6.63066547811636]",Grand Chêne,la


In [12]:
merged = pd.merge(cadastre_exploded, recensement_exploded, left_on = ["owner_names", "street"], right_on = ["PROPRIÉTAIRES DES MAISONS.", "NOMS DES RUES."], how="inner")
print(len(merged))
print(len(merged.drop_duplicates(["nr", "folio"])))

3820
717


In [13]:
dataset = merged[["nr", "folio", "owner_surname", "pos", "street", "NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME.", "NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE.", "VOCATION.", "use"]].dropna()
dataset.columns = ["nr", "folio","owner", "position", "street", "renter_forname", "renter_lastname", "job", "use"]
dataset.position = dataset.position.apply(lambda pos : [float(p) for p in string_to_list(pos)])
print(len(dataset.loc[dataset.query("use == 'maison'").drop_duplicates(["folio", "nr"]).index]) / len(dataset.query("use == 'maison'")))
len(dataset)

0.17601683029453016


3707

In [14]:
dataset.to_json("website/data.json", orient="records")

In [15]:
cadastre.query("owner_surname == 'Jeanne-Françoise Guisan née Dapples'")

,Unnamed: 0,Unnamed: 0.1,folio,nr,articles,use,owner_surname,Unnamed: 6,pos,street,owner_names
1153,1153,1111,6,151,NaN,maison,Jeanne-Françoise Guisan née Dapples,NaN,"[46.52337610743361, 6.635635174087884]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1154,1154,1112,6,152,NaN,cour,Jeanne-Françoise Guisan née Dapples,NaN,"[46.52331763087088, 6.6356203098065585]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1155,1155,1112,6,152,NaN,cour,Jeanne-Françoise Guisan née Dapples,NaN,"[46.523923087883446, 6.634665356724987]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1156,1156,1113,6,153,NaN,maison,Jeanne-Françoise Guisan née Dapples,NaN,"[46.523267938733994, 6.635504033866326]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1157,1157,1114,6,154,NaN,buanderie,Jeanne-Françoise Guisan née Dapples,NaN,"[46.523191099868434, 6.6353881615549755]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1158,1158,1115,6,155,NaN,cour,Jeanne-Françoise Guisan née Dapples,NaN,"[46.523176124028616, 6.635477865389208]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
